In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hdbscan

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score

plt.rcParams["figure.figsize"] = (7, 5)

# =======================================
# 1. LOAD DATA AND PREPROCESS
# =======================================
jobs = pd.read_csv("fake_job_postings.csv")

text_cols = [
    "title","location","department","company_profile","description",
    "requirements","benefits","employment_type","required_experience",
    "required_education","industry","function"
]

jobs[text_cols] = jobs[text_cols].fillna("")
jobs["text"] = jobs[text_cols].agg(" ".join, axis=1).str.lower()

# =======================================
# 2. TF-IDF
# =======================================
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=20000,
    ngram_range=(1,2)
)

X_tfidf = tfidf.fit_transform(jobs["text"])

# =======================================
# 3. SAFE SILHOUETTE FUNCTION
# =======================================
def safe_silhouette(X, labels, name=""):
    mask = labels != -1
    if mask.sum() < 2:
        print(f"[{name}] Too few non-noise points")
        return None
    if len(np.unique(labels[mask])) < 2:
        print(f"[{name}] Only one cluster")
        return None
    score = silhouette_score(X[mask], labels[mask])
    print(f"[{name}] Silhouette: {score:.3f}")
    return score

# =======================================
# 4. HYPERPARAMETER SEARCH
# =======================================
svd_components_list = [100, 200, 300,400,500]  # adjust as needed
min_cluster_size_list = [20, 30, 50]
min_samples_list = [5, 10]

results = []

best_score = -1
best_model = None
best_labels = None
best_X_red = None
best_params = None

for k in svd_components_list:
    print(f"\n=== Testing SVD components = {k} ===")

    svd = TruncatedSVD(n_components=k, random_state=42)
    X_red = svd.fit_transform(X_tfidf)

    for mcs in min_cluster_size_list:
        for ms in min_samples_list:

            clusterer = hdbscan.HDBSCAN(
                min_cluster_size=mcs,
                min_samples=ms
            )
            labels = clusterer.fit_predict(X_red)

            score = safe_silhouette(X_red, labels,
                                   name=f"SVD={k}, mcs={mcs}, ms={ms}")

            results.append({
                "svd_components": k,
                "min_cluster_size": mcs,
                "min_samples": ms,
                "silhouette": score,
                "num_clusters": len(np.unique(labels)),
                "num_noise": np.sum(labels == -1)
            })

            if score is not None and score > best_score:
                best_score = score
                best_model = clusterer
                best_labels = labels
                best_X_red = X_red
                best_params = (k, mcs, ms)

# =======================================
# 5. SHOW BEST RESULT
# =======================================
print("\n==============================")
print("BEST HDBSCAN CONFIGURATION")
print("==============================")
print(f"SVD components: {best_params[0]}")
print(f"min_cluster_size: {best_params[1]}")
print(f"min_samples: {best_params[2]}")
print(f"Best silhouette: {best_score:.3f}")

# =======================================
# 6. VISUALIZE BEST MODEL IN 2D
# =======================================
X_2d = best_X_red[:, :2]

plt.figure()
unique_labels = np.unique(best_labels)
for lab in unique_labels:
    mask = best_labels == lab
    name = f"Cluster {lab}" if lab != -1 else "Noise"
    plt.scatter(X_2d[mask, 0], X_2d[mask, 1], s=10, alpha=0.6, label=name)

plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.title(f"HDBSCAN Clusters (SVD={best_params[0]})")
plt.legend(markerscale=2, fontsize=8)
plt.show()

# =======================================
# 7. ATTACH LABELS TO DATAFRAME
# =======================================
jobs["hdbscan_cluster"] = best_labels
jobs["hdbscan_noise"] = (best_labels == -1).astype(int)

print("\nCluster counts:")
print(jobs["hdbscan_cluster"].value_counts().sort_index())



=== Testing SVD components = 50 ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[SVD=50, mcs=20, ms=5] Silhouette: 0.171


/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[SVD=50, mcs=20, ms=10] Silhouette: 0.222


...

/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[SVD=200, mcs=30, ms=5] Silhouette: 0.538


/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[SVD=200, mcs=30, ms=10] Silhouette: 0.521


/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
